## Multiprocessing

$\textbf{Borrowed from}$ 

$ \textit{codebasics}$<br>
https://www.youtube.com/playlist?list=PLeo1K3hjS3uub3PRhdoCTY8BxMKSW7RjN

In [2]:
import time
import multiprocessing

### Snippet 1  <br>$\textit{Hello World}$

In [104]:
arr = [i for i in range(5)]
def getSquare(x):
    for i in x:
        time.sleep(2)
        print('', end='s ')
    print()
    
    
def getCube(x):
    for i in x:
        time.sleep(3)
        print('', end='c ')
    print()
    
p1 = multiprocessing.Process(target = getSquare, args = (arr,))
p2 = multiprocessing.Process(target = getCube, args = (arr,))

p1.start()
p2.start()


p1.join()
p2.join()

print('Done')

s s s s s 
c c c c c 
Done


### Snippet 2 <br>$\textit{Shortcoming of global variables}$

In [107]:
square_result = []
arr = [i for i in range(5)]
def getSquare(x):
    global square_result
    for i in x:
        square_result.append(i*i)
    
    
    
p1 = multiprocessing.Process(target = getSquare, args = (arr,))

p1.start()
p1.join()

print(square_result)
#Global variables are not shared amongst processes

[]


### Snippet 3 <br>$\textit{Array}$

In [108]:
#Array as shared variable
square_result = multiprocessing.Array('i',5) #Here i stands for integer. For double it is d
arr = [i for i in range(5)]

def getSquare(x, square_result):
    for idx, item in enumerate(x):
        square_result[idx] = item**2
    
    
    
p1 = multiprocessing.Process(target = getSquare, args = (arr, square_result))
p1.start()
p1.join()


print(square_result[:])

[0, 1, 4, 9, 16]


### Snippet 4 <br>$\textit{Value}$

In [7]:
#Value as shared Variable
val = multiprocessing.Value('d',0.5)
arr = [i for i in range(5)]

def getSquare(x, val):
    for item in x:
        val.value+=(item**2)
    
    
    
p1 = multiprocessing.Process(target = getSquare, args = (arr, val))
p1.start()
p1.join()


print(val.value)

30.5


### Snippet 5 <br>$\textit{Queue}$

In [34]:
#Queue as shared Variable
# q = multiprocessing.Queue()
arr = [i for i in range(5)]

def getSquare(x, q):
    for item in x:
        time.sleep(2)
        q.put('s')
    
def getCube(x, q):
    for item in x:
        time.sleep(3)
        q.put('c')
        
#The time intervals have been chosen and are not random
    
p1 = multiprocessing.Process(target = getSquare, args = (arr, q))
p2 = multiprocessing.Process(target = getCube, args = (arr, q))
p1.start()
p2.start()

p1.join()
p2.join()



while q.empty() is False:
    print(q.get(),end=' ')

s c s s c s c s c c 

### Snippet 6  <br>$\textit{Locks}$

In [105]:
#Without lock
def deposit(balance):
    for i in range(100):
        time.sleep(0.01)
        balance.value+=1
        
def withdraw(balance):
    for i in range(100):
        time.sleep(0.02)
        balance.value-=1
        


for i in range(20):
    balance = multiprocessing.Value('i',200)
    d = multiprocessing.Process(target = deposit, args = (balance,))
    w = multiprocessing.Process(target = withdraw, args = (balance,))
    d.start()
    w.start()
    d.join()
    w.join()
    
    print(balance.value, end=' ')

200 200 200 200 200 200 200 200 200 200 200 199 200 200 200 200 200 200 199 200 

In [56]:
#With lock
def deposit(balance, lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value+=1
        lock.release()
        
def withdraw(balance, lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value-=1
        lock.release()
        

for i in range(20):
    balance = multiprocessing.Value('i',200)
    lock = multiprocessing.Lock()
    d = multiprocessing.Process(target = deposit, args = (balance, lock))
    w = multiprocessing.Process(target = withdraw, args = (balance, lock))
    d.start()
    w.start()
    w.join()
    d.join()
    print(balance.value,end=' ')

200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 

### Snippet 6 <br>$\textit{Divide among cores}$

In [ ]:
from multiprocessing import Pool

In [91]:
def f(n):
    return n**2

p = Pool()

a = time.time()
result = p.map(f, range(5)) #This alone will divide the work among all available cores equally
p.close()
p.join()
b = time.time() - a
print(result)

[0, 1, 4, 9, 16]


In [99]:
def f(n):
    sum = 0
    for i in range(10000):
        sum+=(i*i)
    

p = Pool()

a = time.time()
result = p.map(f, range(10000)) #This alone will divide the work among all available cores equally
p.close()
p.join()
b = time.time() - a
print('Time taken: {}'.format(b))


Time taken: 1.8142108917236328


In [101]:
a = time.time()
for i in range(10000):
    sum = 0
    for j in range(10000):
        sum+=(j*j)

b = time.time()- a
print('Time taken: {}'.format(b))





Time taken: 10.611477613449097


Number of cores

In [66]:
multiprocessing.cpu_count()

8